In [1]:
"""
Some results (e.g. miconic evaluation, or blocksworld training) are either timing out (due to apptainer and cluster cpus) or running out of memory (possibly due to suboptimal handcrafted policies, or very large training datasets).
"""

from _results_helper import *

this_file_dir=PosixPath('/home/dzc/code/cvut-colab/results')
Raw logs found. Rewriting data to csv files...


/home/dzc/code/cvut-colab/results/baseline_logs/transport_1_30_r4.log: 100%|██████████| 1200/1200 [00:00<00:00, 3850.38it/s]


skipping hard problems because they take too long


# Baseline Cycle Count

In [2]:
visualise_cylces()

Ideally, we want 0 cycles


# Plan length plots

In [3]:
log_y = 0
choices = [
    # "sample",
    "best",
]
layers = [
    1,
    2,
    3,
    # 4,
    # 5,
]
dimensions = [
    1,
    # 2,
    # 4,
    # 8,
    16,
    32,
    64,
]

plot_domains("plan_length", log_y=log_y, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


transport


In [4]:
plot_domains("cycles", log_y=log_y, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


transport


# Absolute difference plots

In [5]:
plot_difference(absolute=True, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


transport


# Relative difference plots

In [6]:
plot_difference(absolute=False, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


transport
